## Testing path length as an accuracy measure

Quick notebook to test the validity of this idea.

In [1]:
import sys
sys.path.append('./source/') # append files from source folder

import torch
from GenerateShortestPathData import create_shortest_path_data
from ModelsShortestPath import ShortestPathNet, Cvx_ShortestPathNet, Pert_ShortestPathNet, BB_ShortestPathNet
import matplotlib.pyplot as plt
from utils import Regret
import time as time
from Trainer import trainer
import numpy as np

## Set device
device = 'cuda:0'

In [2]:
## Some fixed hyperparameters
max_epochs = 100
init_lr = 1e-2 # initial learning rate. We're using a scheduler.
torch.manual_seed(0)
device = 'cuda:0'

In [3]:
## Some arrays
tl_trained_DYS = [] # test loss
tt_trained_DYS = [] # training time
ta_trained_DYS = [] # test accuracy of trained model
ne_trained_DYS = [] # number of epochs completed during training

tl_trained_CVX = []
tt_trained_CVX = []
ta_trained_CVX = []
ne_trained_CVX = []

tl_trained_PertOpt = []
tt_trained_PertOpt = []
ta_trained_PertOpt = []
ne_trained_PertOpt = []

tl_trained_BB = []
tt_trained_BB = []
ta_trained_BB = []
ne_trained_BB = []

In [4]:
for grid_size in range(5, 6, 5):
    
    ## Load data
    data_path = './shortest_path_data/Shortest_Path_training_data'+str(grid_size)+'.pth'
    state = torch.load(data_path)

    ## Extract data from state
    train_dataset_e = state['train_dataset_e']
    test_dataset_e = state['test_dataset_e']
    train_dataset_v = state['train_dataset_v']
    test_dataset_v = state['test_dataset_v']
    m = state["m"]
    A = state["A"].float()
    WW = state["WW"].float()
    b = state["b"].float()
    num_edges = state["num_edges"]
    Edge_list = state["Edge_list"]
    Edge_list_torch = torch.tensor(Edge_list)
    
    ## Set device
    device = 'cuda:0'
    
#     ## Load model/network
#     DYS_net = ShortestPathNet(A, b, num_vertices = grid_size**2, num_edges = num_edges ,
#                       Edges = Edge_list_torch.to(device), context_size = 5)
#     DYS_net.to(device)
    
#     # Train
#     tl_DYS, tt_DYS, ta_DYS = trainer(DYS_net, train_dataset_e, test_dataset_e, grid_size, WW, 
#                                     max_epochs, init_lr, graph_type='E', Edge_list = Edge_list)
    
    ## Load model/network
#     CVX_net = Cvx_ShortestPathNet(A.float(), b.float(), 5)
#     CVX_net.to(device)


#     # Train
#     print('\n-------------------------------------------- TRAINING CVX GRID ' + str(grid_size) + '-by-' + str(grid_size) + ' --------------------------------------------')
#     start_time = time.time()
#     tl_CVX, tt_CVX, ta_CVX = trainer(CVX_net, train_dataset_e, test_dataset_e, grid_size, WW,
#                             max_epochs, init_lr, graph_type='E', Edge_list = Edge_list, max_time=np.inf)

#     end_time = time.time()
#     print('\n time to train CVX GRID ' + str(grid_size) + '-by-' + str(grid_size), ' = ', end_time-start_time, ' seconds')
    
    ## Load model/network
    PertOpt_net = Pert_ShortestPathNet(grid_size, context_size=5, device='cpu') # PertOpt not GPU Compatible
    PertOpt_net.to('cpu')

    # Train
    print('\n-------------------------------------------- TRAINING PertOpt GRID ' + str(grid_size) + '-by-' + str(grid_size) + ' --------------------------------------------')
    start_time = time.time()
    tl_PertOpt, tt_PertOpt, ta_PertOpt = trainer(PertOpt_net, train_dataset_v,
                                              test_dataset_v, grid_size, WW, max_epochs,
                                              init_lr, graph_type='V', Edge_list = Edge_list,
                                              device='cpu', max_time=np.inf, use_scheduler=False) # note PertOpt is not GPU compatible
     


-------------------------------------------- TRAINING PertOpt GRID 5-by-5 --------------------------------------------


/usr/lib/python3/dist-packages/torch/optim/lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


epoch:  0 | ave_tr_loss:  5.57e-02 | te_loss:  3.16e-01 | acc.:  0.000000 | lr:  1.00e-02 | regret:  0.274905 | time:  8.167840       
epoch:  1 | ave_tr_loss:  9.47e-02 | te_loss:  3.07e-01 | acc.:  0.020000 | lr:  1.00e-02 | regret:  0.113401 | time:  16.907581      
epoch:  2 | ave_tr_loss:  1.22e-01 | te_loss:  2.75e-01 | acc.:  0.370000 | lr:  1.00e-02 | regret:  0.037209 | time:  25.595192      
epoch:  3 | ave_tr_loss:  1.43e-01 | te_loss:  2.75e-01 | acc.:  0.370000 | lr:  1.00e-02 | regret:  0.037209 | time:  34.205335      
epoch:  4 | ave_tr_loss:  1.57e-01 | te_loss:  2.75e-01 | acc.:  0.370000 | lr:  1.00e-02 | regret:  0.037209 | time:  42.789703      
epoch:  5 | ave_tr_loss:  1.66e-01 | te_loss:  2.73e-01 | acc.:  0.375000 | lr:  1.00e-02 | regret:  0.036119 | time:  51.348838      
epoch:  6 | ave_tr_loss:  1.71e-01 | te_loss:  2.64e-01 | acc.:  0.385000 | lr:  1.00e-02 | regret:  0.032562 | time:  59.883339      
epoch:  7 | ave_tr_loss:  1.73e-01 | te_loss:  2.10e-01

KeyboardInterrupt: 

In [ ]:
from torch.utils.data import DataLoader
device = 'cuda:0'
WW = state["WW"].float()
WW = WW.to(device)
test_loader_e = DataLoader(dataset=test_dataset_e, batch_size=200, shuffle=False)
d_batch, path_batch = next(iter(test_loader_e))
d_batch = d_batch.to(device)
path_batch = path_batch.to(device)
pred_batch = DYS_net(d_batch)

In [ ]:
print(WW)

In [ ]:
regret = Regret(WW,d_batch, path_batch, pred_batch)

In [ ]:
print(regret)